In [1]:
#Need these packages and dependecies, can access these by launching jupyter from within the legend-base.sif container from the readme

from lgdo.lh5 import LH5Store, ls, show, load_dfs

from dspeed.vis import WaveformBrowser
from dspeed import units
from dspeed import build_dsp
import numpy as np


from scipy import stats


import matplotlib.pyplot as plt
import os , json


import pint
u = pint.get_application_registry()


import os

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 12



#Path to all of the output files from the simulation
#Note the range depends on whether on how many threads and jobs you submit in an array

data_files = [f"/path/to/output_file{j}_t{i}.lh5" for j in range(1, 11) for i in range(0, 10)]

show(data_files[99])


/
└── stp · struct{det1078400,det1078401,det1078402,det1078403,det1078404,det1078405,det1080000,det1080001,det1080002,det1080003,det1080004,det1080005,det1081600,det1081601,det1104004,det1104005,det1105600,det1105602,det1107200,det1107201,det1107202,det1107203,det1107205,det1108800,det1108801,det1108802,det1108803,det1108804,det1108805,det1110400,det1110401,det1110402,det1110403,det1110404,det1110405,det1112000,det1112001,det1112002,det1112003,det1112004,det1113600,det1113603,det1113604,det1113605,det1115203,det1115204,det1115205,det1116801,det1116802,det1118400,det1118401,det1118402,det1118404,det1118405,det1120000,det1120001,det1120002,det1120004,det1120005,det1121600,vertices} 
    ├── det1078400 · table{evtid,particle,trackid,parent_trackid,edep,time,xloc,yloc,zloc,dist_to_surf,xloc_pre,yloc_pre,zloc_pre,dist_to_surf_pre,xloc_post,yloc_post,zloc_post,dist_to_surf_post} 
    │   ├── dist_to_surf · array<1>{real} 
    │   ├── dist_to_surf_post · array<1>{real} 
    │   ├── dist_to_su

In [2]:
import re

#This is a just a way for me to get all the detectors for the stp file, it is annoying but whatever
command_text = open("detector_list.txt").read()


detector_ids = re.findall(r'/RMG/Geometry/RegisterDetector Germanium \S+ (\d+)', command_text)
detector_paths = [f"stp/det{id}" for id in detector_ids]

print(detector_paths)

['stp/det1104004', 'stp/det1104005', 'stp/det1105600', 'stp/det1107200', 'stp/det1107201', 'stp/det1107202', 'stp/det1107203', 'stp/det1107205', 'stp/det1108800', 'stp/det1108801', 'stp/det1108802', 'stp/det1105602', 'stp/det1108803', 'stp/det1108804', 'stp/det1108805', 'stp/det1110400', 'stp/det1110401', 'stp/det1110402', 'stp/det1110403', 'stp/det1110404', 'stp/det1110405', 'stp/det1112000', 'stp/det1112001', 'stp/det1112002', 'stp/det1112003', 'stp/det1112004', 'stp/det1113600', 'stp/det1113603', 'stp/det1113604', 'stp/det1113605', 'stp/det1118401', 'stp/det1115204', 'stp/det1115205', 'stp/det1116801', 'stp/det1116802', 'stp/det1115203', 'stp/det1118400', 'stp/det1118402', 'stp/det1118404', 'stp/det1118405', 'stp/det1120000', 'stp/det1120001', 'stp/det1120002', 'stp/det1120004', 'stp/det1120005', 'stp/det1121600', 'stp/det1078400', 'stp/det1078401', 'stp/det1078402', 'stp/det1078403', 'stp/det1078404', 'stp/det1078405', 'stp/det1080000', 'stp/det1080001', 'stp/det1080002', 'stp/det1

In [6]:
from lgdo.lh5 import LH5Store, ls, show, load_dfs

from dspeed.vis import WaveformBrowser
from dspeed import units
from dspeed import build_dsp
import numpy as np

from scipy import stats

import matplotlib.pyplot as plt
import os, json

import pint
u = pint.get_application_registry()

import glob
import pandas as pd

# Files to process
files = sorted(glob.glob("/path/to/output_file{j}_t{i}.lh5"))

store = LH5Store()
fcp_detector_summary = []

for f in files:
    print(f"Processing file: {f}")
    for path in detector_paths:
        try:
            #This sorts through all the detectors in the lh5 output file, I did not write this spcecific line of code
            obj, _ = store.read(path, f)
            df = pd.DataFrame({k: v.nda for k, v in obj.items()})

            if df.empty:
                continue

            #FCP's only have a PID of +/- 90 so that is all I am keeping
            #Note if you want to do muons the number is different, 13 I think
            #Another note, if you want to study secondary showers comment line below out all together
            
    
            df = df[df["particle"].abs() == 90]

            if df.empty:
                continue  #This is a sanity check and goes away with proper statisitcs but still good to check

            #This created a new column which is the detector that the particle crosses, it gets this from the file name
            df["detector_id"] = int(path.split("det")[-1])
            
            #I could delete this but it was a sanity check, just saves the output file 
            df["file"] = f

            #This groups by event id (each particle) and takes the length of each particle step and sums them up for total path length
            #Keep in mind this whole thing is done on an individual detector basis so we are not summing for total path length
            #Output is in meters for god knows what reason so changing it to mm
            for evtid, df_evt in df.groupby("evtid"):
                path_len = np.sqrt(
                    (df_evt["xloc_post"] - df_evt["xloc_pre"])**2 +
 
                    
                    (df_evt["yloc_post"] - df_evt["yloc_pre"])**2 +
                    (df_evt["zloc_post"] - df_evt["zloc_pre"])**2
                ).sum() * 1000

                #This just sums all energy depositions by a particle to get total edep
                total_edep = df_evt["edep"].sum()

                #Sorts it by time to get the first "initial" hit, will use this to get solid angle coverage later on
                df_sorted = df_evt.sort_values("time")
                first_hit = df_sorted.iloc[0]

                #This saves the event id to the summary, detector hit, file name, path length, edep, total hits, number of hits (which is just the number of 
                #rows for a particle and number of steps), the time at which the particle has its first hit as well as the location of the first hit
                fcp_detector_summary.append({
                    "evtid": evtid,
                    "detector_id": int(path.split("det")[-1]),
                    "file": f,
                    "path_length": path_len,
                    "total_edep": total_edep,
                    "n_hits": len(df_evt),
                    "entry_time": first_hit["time"],
                    "x_entry": first_hit["xloc"],
                    "y_entry": first_hit["yloc"],
                    "z_entry": first_hit["zloc"],
                })
        #File Path error message
        except Exception as e:
            print(f"Failed to read {path} in {f}: {e}")
            continue

df_summary_raw = pd.DataFrame(fcp_detector_summary)
print(df_summary_raw.head())


#Output file is by particle by detector data. So there are multiple rows with the same evtid, detector etc. 

Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t0.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t1.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t2.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t3.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t4.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t5.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t6.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t7.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t8.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV10_t9.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV1_t0.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV1_t1.lh5
Processing file: LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_115_100GeV1_t2.lh5
Processing file: LH5 Data/FCP_Plane_P15/F

In [ ]:
import re
import pandas as pd

# Work on a copy so you don’t get a “SettingWithCopy” warning later
df = df_summary_raw.copy()

# 1. FIX the stray “prin” (looks like a half‑typed print).
# 2. Define the extractor
def extract_job_number(fname: str) -> int:
    """
    Grab the integer that appears after 'GeV' and before '_t' in the file name.
    Example:  ...100GeV10_t3.lh5  ->  10
    """
    #Change M back to G
    m = re.search(r'GeV(\d+)_t', fname)
    if m:
        return int(m.group(1))
    else:
        raise ValueError(f"Could not parse job number from {fname}")

# 3. Vectorised extraction
df["job_number"] = df["file"].apply(extract_job_number).astype(int)

# 4. Compute globally unique event IDs, note this depends on how many particles you simulate for each job in paralell

#Note this value will change depending on how many particles you simulate for each job
df["global_evtid"] = df["evtid"] + (df["job_number"] - 1) * 10000000


# (Optional) quick sanity check
print(
    df[["file", "evtid", "job_number", "global_evtid"]]
    .head()
    .to_string(index=False)
)


In [ ]:
#This renames the evtid column to local evtid or the evtid within that respective job
#Global_evtid becomes normal evtid which can be used now regardless of which job in the array simulated it



df = df.rename(columns={
    "evtid": "local_evtid",
    "global_evtid": "evtid"
})

print(df.head(20))


#Save the "raw" file, this is without cuts or anything like that. This is important for later 
df.to_pickle("df_summary_raw.pkl")

In [ ]:
#Period 8 Inactive Detectors

#This is how I screen out period 8, inactive detectors. Or just inactive detectors in general
#Note I do it this way because I don't have to run multiple simulations by flipping detectors on and off

import pandas as pd
df_summary = pd.read_pickle("df_summary_raw.pkl")


detector_locations = pd.read_excel("L200_Det_Radii_Pos.xlsx")


#This is where I put in all the inactive detectors

banned_patterns = ["V07298B" ,"P00665A", "V01386A", "V01403A", "V01404A", "V05261A", "B00091D", "P00537A", 
                   "P00538B", "P00661A", "P00665B", "P00698B","C00ANG3", "C00ANG5", "C00ANG2", "V00074A", "P00712A", "V01406A", "V01415A", 
                   "V05267B", "B00089D", "V01387A", "B00002A", "P00748B", "B00091B"]



banned_det_ids = detector_locations[
    detector_locations["Detector"].isin(banned_patterns)
]["Detector_ID"].unique()


#This removes all the hits and data from the inactive detectors
df_summary = df_summary[~df_summary["detector_id"].isin(banned_det_ids)]





print(df_summary.head(20))

In [187]:
#This is for P08, active realistic threshold drop to noise 


#This is how I screen out period 8, inactive detectors. Or just inactive detectors in general
#Note I do it this way because I don't have to run multiple simulations by flipping detectors on and off





import pandas as pd


df_summary = pd.read_pickle("df_summary_raw.pkl")


detector_locations = pd.read_excel("L200_Det_Radii_Pos.xlsx")


#Inactive Detectors
banned_patterns = ["V07298B" ,"P00665A", "V01386A", "V01403A", "V01404A", "V05261A", "B00091D", "P00537A", 
                   "P00538B", "P00661A", "P00665B", "P00698B","C00ANG3", "C00ANG5", "C00ANG2", "V00074A", "P00712A", "V01406A", "V01415A", 
                   "V05267B", "B00089D", "V01387A", "B00002A", "P00748B", "B00091B"]


banned_det_ids = detector_locations[
    detector_locations["Detector"].isin(banned_patterns)
]["Detector_ID"].unique()


#Keeps only active detectors
df_summary = df_summary[~df_summary["detector_id"].isin(banned_det_ids)]



#NEED to group these particles by EVTID, then sort by time

#find the First detector that is "over 40 keV" in the sequence and keep all the detectors after it that meet threshold, 10 keV for COAX, 6 keV for the rest

#C000RG1 1107203
#C000RG2 1107204
#C00ANG3 1107205
#C00ANG5 1108800
#C00ANG2 1108801
#C00ANG4 1120005


#Coax detectors have different energy threshold value so they get special treatement

coax_ids = {
    "1107203", "1107204",
    "1107205", "1108800",
    "1108801", "1120005"
}

def apply_thresholds(df):
    # Sort by decreasing z_entry instead of time because I do not have it, should still give progression of detectors hit
    df = df.sort_values("z_entry", ascending=False).reset_index(drop=True)

    #Filters for all the 40 keV edeps for the particle
    idx = df.index[df["total_edep"] >= 40]
    if len(idx) == 0:
        return pd.DataFrame([])  # drop whole event if none are above 40 keV

    #This grabs the index of the first > 40 keV hit
    start_idx = idx[0]

    # keeps all hits after the first 40 keV
    df = df.loc[start_idx:].copy()

    # this applies a 10 keV cut for all events in Coax and a 6 keV cut for all non-coax events
    is_coax = df["detector_id"].isin(coax_ids)
    df = df[
        (is_coax & (df["total_edep"] >= 10)) |
        (~is_coax & (df["total_edep"] >= 6))
    ]

    return df

#Groups per particle, and applies threshold cuts that way

df_summary = (
    df_summary
    .groupby("evtid", group_keys=False)
    .apply(apply_thresholds)
    .reset_index(drop=True)
)




#Thus is how you find the multiplicity values, 
#For each evtID, how many unique detectors_id's are there


#Groups everything by evtid, and counts number of unique detector_ids
multiplicity_map = df_summary.groupby("evtid")["detector_id"].nunique()

#This maps it to evtid so each evtid as a multiplicity value
df_summary["multiplicity"] = df_summary["evtid"].map(multiplicity_map)

#New Data File
df_summary.to_pickle("df_summary_cut40.pkl")



     local_evtid  detector_id  \
49        145710      1105600   
52        210914      1105600   
102       395071      1107202   
114       232161      1107203   
117       415514      1107203   
136       239195      1108800   
144       485928      1108800   
256       345134      1108805   
266        70267      1110400   
346       381522      1110405   
355        91121      1112000   
442       288871      1113603   
443       303509      1113603   
561         6150      1115203   
577       153194      1118400   
599       499691      1118402   
641       160245      1120001   
650         5774      1120002   
653        70109      1120002   
655        77343      1120002   

                                                  file  path_length  \
49   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_40_100GeV...   117.811222   
52   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_40_100GeV...   113.307584   
102  LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_40_100GeV...    66.467516   
114  LH5 Data/FCP_Plan

In [188]:
angle_rows = []

#This goes through and identifies the file each particle row has.
for f in df_summary["file"].unique():

    df_summary_file = df_summary[df_summary["file"] == f].copy()

    #Using the file in the output from earlier it finds the file and gets the vertex information
    df_vertex = load_dfs([f], ["evtid", "xloc", "yloc", "zloc"], "stp/vertices")

    # Filter to only relevant events
    df_vertex = df_vertex[df_vertex["evtid"].isin(df_summary_file["local_evtid"])]

    # Rename for clarity
    df_vertex = df_vertex.rename(columns={
         "evtid": "local_evtid",
         "xloc": "x_init",
         "yloc": "y_init",
         "zloc": "z_init"
     })

    # Merge vertex into summary by evtid
    merged = pd.merge(df_summary_file, df_vertex[["local_evtid", "x_init", "y_init", "z_init"]], on="local_evtid", how="left")

    # Find the direction vector
    dx = merged["x_init"] - merged["x_entry"]
    dy = merged["y_init"] - merged["y_entry"]
    dz = merged["z_init"] - merged["z_entry"]
    P = np.sqrt(dx**2 + dy**2)

    # Calculate the angle of entry
    merged["theta"] = np.arctan2(P, dz)
    merged["phi"] = np.arctan2(dy, dx)

    # Save 
    angle_rows.append(merged)

# Combine all chunks into updated df_summary 
df_summary = pd.concat(angle_rows, ignore_index=True)

print(df_summary.head(20))

#New file has multiplicity, the intial hit in each detector, the intial starting position of the particle, and the angle that defines the momentum

/tmp/ipykernel_181719/3814645739.py:9: DeprecationWarning: load_dfs() is deprecated. Please replace it with LH5Store.read(...).view_as('pd'), or just read_as(..., 'pd'). load_dfs() will be removed in a future release.
  df_vertex = load_dfs([f], ["evtid", "xloc", "yloc", "zloc"], "stp/vertices")
/tmp/ipykernel_181719/3814645739.py:9: DeprecationWarning: load_dfs() is deprecated. Please replace it with LH5Store.read(...).view_as('pd'), or just read_as(..., 'pd'). load_dfs() will be removed in a future release.
  df_vertex = load_dfs([f], ["evtid", "xloc", "yloc", "zloc"], "stp/vertices")
/tmp/ipykernel_181719/3814645739.py:9: DeprecationWarning: load_dfs() is deprecated. Please replace it with LH5Store.read(...).view_as('pd'), or just read_as(..., 'pd'). load_dfs() will be removed in a future release.
  df_vertex = load_dfs([f], ["evtid", "xloc", "yloc", "zloc"], "stp/vertices")
/tmp/ipykernel_181719/3814645739.py:9: DeprecationWarning: load_dfs() is deprecated. Please replace it with L

    local_evtid  detector_id  \
0         26897      1104005   
1         97185      1104005   
2        198750      1104005   
3        219170      1104005   
4        219256      1104005   
5        240883      1104005   
6        268165      1104005   
7        281408      1104005   
8        302681      1104005   
9        345526      1104005   
10       387580      1104005   
11       435024      1104005   
12       485264      1104005   
13        75947      1105600   
14        90273      1105600   
15        97185      1105600   
16       110147      1105600   
17       118001      1105600   
18       156784      1105600   
19       185153      1105600   

                                                 file  path_length  \
0   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_30_100GeV...   104.662785   
1   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_30_100GeV...   101.331862   
2   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_30_100GeV...    95.093126   
3   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_30_100

In [189]:
#Sanity check to see what the average angles are for each 'm' value


#Now we have our desired output 

avg_theta = df_summary.groupby("multiplicity")["theta"].mean()

avg_phi = df_summary.groupby("multiplicity")["phi"].mean()

print(avg_theta)

print(df_summary.head(20))

multiplicity
1    0.805728
2    0.746319
3    0.653354
4    0.496665
5    0.372472
6    0.290164
7    0.210853
8    0.188261
Name: theta, dtype: float64
    local_evtid  detector_id  \
0         26897      1104005   
1         97185      1104005   
2        198750      1104005   
3        219170      1104005   
4        219256      1104005   
5        240883      1104005   
6        268165      1104005   
7        281408      1104005   
8        302681      1104005   
9        345526      1104005   
10       387580      1104005   
11       435024      1104005   
12       485264      1104005   
13        75947      1105600   
14        90273      1105600   
15        97185      1105600   
16       110147      1105600   
17       118001      1105600   
18       156784      1105600   
19       185153      1105600   

                                                 file  path_length  \
0   LH5 Data/FCP_Plane_P15/FCP_PLANE_P15_30_100GeV...   104.662785   
1   LH5 Data/FCP_Plane_P15/FCP_PLA

In [ ]:

#Now to calculate the radii of each detector hit

import pandas as pd
df_summary = pd.read_pickle("df_summary_cut40.pkl")

#File that has all the Radii of the l200 detectors
#Has two columns, "Detector_ID" and "Radius(mm)"
radii_df = pd.read_excel("L200_Det_Radii_Pos.xlsx")
print(radii_df.columns.tolist())

#I dont want the positions of the detectors for this 
columns_to_keep = ["Detector_ID", "Detector", "Radius (mm)"]

radii_df = radii_df[columns_to_keep]

#Now all of the detectors radii is merged into the big data file

merged_df = df_summary.merge(
    radii_df,
    left_on="detector_id",
    right_on="Detector_ID",
    how="left"
)





print(merged_df.head())





In [ ]:


#Converts everything to Area in cm^2

merged_df["Area_cm2"] = (np.pi * merged_df["Radius (mm)"]**2)/100


merged_df = merged_df.drop(columns=["Radius (mm)", "Detector_ID"])

print(merged_df.head())

In [ ]:
# Find the index of the row with the smallest Area_cm2 for each evtid
min_area_idx = merged_df.groupby("evtid")["Area_cm2"].idxmin()

#min_area_idx = min_area_idx.dropna().astype(int)

# Use those indices to filter the original DataFrame
smallest_area_df = merged_df.loc[min_area_idx].reset_index(drop=True)


In [ ]:
#Takes the smallest area hit per each event, and takes the mean for each multiplicty value
mean_area = smallest_area_df.groupby('multiplicity')['Area_cm2'].mean()

print(mean_area)

#Converts to a .pkl file we will use later on
mean_area.to_pickle("area_40.pkl")


